# **Canvas API Exploration Notebook**

### **Step 0. Run ***pip install -r requirements.txt*** in your Terminal** 
#### (you may need to include either a "python" or "python -m" prefix)

### **Step 1. Import Required Libraries**

In [1]:
# canvasapi used as API Wrapper
# Pandas used as Data Exploration and Manipulation Tool

from canvasapi import Canvas
import pandas as pd

/tmp/ipykernel_16434/2531346552.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### **Step 2. Initialize Constants and Master Canvas Object**

In [2]:
# Canvas API URL
API_URL = "https://uvu.instructure.com"

# Canvas API key (follow instructions on README file)
API_KEY = '1012~cx4N6JNIB7wiK64srwnI9VWx0YdzWz1iZ0IOQikNrEFubdx5GUaM6auXEaXkBiZ6'

# Canvas User ID
USER_ID = 1957033

# Initialize a new Canvas object
canvas = Canvas(API_URL, API_KEY)

### **Step 3. Initialize Lists**

In [3]:
# These lists will act as temporary data stores and be the blueprints for the tables to be used in the Transactional Database

courses = []
course_objects = []
course_assignments = []
course_assignment_objects = []
course_assignment_submissions = []

### **Step 4. Load Data from All Courses into DataFrame Object**

In [4]:
# iterate through all available courses, append the raw strings to a list of dictionaries, append the Course objects to a separate list
for course in canvas.get_courses():
    courses.append({'course_id':course.id, 'course_name':course.name})
    course_objects.append(course)

# output list of dictionaries as Pandas DataFrame
courses = pd.DataFrame(courses)

# view results
courses

,course_id,course_name
0,574675,COMM-1020-002 | 2024 Spring - Full Term
1,596443,CS-1400-002-006-Spring 2024-XLIST | Kuo
2,568542,CS-1400-008 | 2023 Fall - Full Term
3,565336,DGM-2130-001 | 2023 Fall - Full Term
4,579706,DGM-2460-001 | 2024 Spring - Full Term
5,572108,DWDD-1400-601 | 2023 Fall - Full Term
6,567415,MATH-1060-007 | 2023 Fall - Full Term
7,574491,MATH-1210-X01 | 2024 Spring - Full Term
8,565624,PHYS-1700-001 | 2023 Fall - Full Term
9,583839,UVU Canvas Tour Fall 2023


### **Step 5. Load All Assignments from All Courses into DataFrame Object**

In [5]:
# OUTER FOR LOOP: iterate through all available courses
# INNER FOR LOOP: for each course, iterate over all available assignments, append full Assignment Objects to list, append the raw strings to a list of dictionaries
for course in course_objects:
    for assignment in course.get_assignments():
        course_assignment_objects.append(assignment)
        course_assignments.append({'course_id':course.id, 'assignment_id':assignment.id, 'assignment_name':assignment.name, 'description':assignment.description, 'submitted':assignment.has_submitted_submissions, 'points_possible':assignment.points_possible, 'submission_types':assignment.submission_types})
        
# output list of dictionaries as Pandas DataFrame
course_assignments = pd.DataFrame(course_assignments)

# view results
course_assignments

,course_id,assignment_id,assignment_name,description,submitted,points_possible,submission_types
0,574675,7415005,Module 1: Student Information and Syllabus Con...,"<link rel=""stylesheet"" href=""https://instructu...",True,10.0,[online_upload]
1,574675,7415009,Module 2: Getting Started: Self-Assessment and...,"<link rel=""stylesheet"" href=""https://instructu...",True,5.0,[online_upload]
2,574675,7414997,Module 16: Culmination: Self-Assessment and Goals,"<link rel=""stylesheet"" href=""https://instructu...",False,5.0,"[online_text_entry, online_upload]"
3,574675,7414999,Module 16: SRI Extra Credit,"<link rel=""stylesheet"" href=""https://instructu...",False,0.0,"[online_text_entry, online_upload]"
4,574675,7415058,Participation,"<link rel=""stylesheet"" href=""https://instructu...",False,100.0,[none]
...,...,...,...,...,...,...,...
466,583839,6975017,Complete: Canvas Quizzes and Exams quiz,"<link rel=""stylesheet"" href=""https://instructu...",True,3.0,[online_quiz]
467,583839,6975023,Submit: Upload a File,"<link rel=""stylesheet"" href=""https://instructu...",True,0.0,[online_upload]
468,583839,6975020,Complete: Grades Quiz,"<link rel=""stylesheet"" href=""https://instructu...",True,6.0,[online_quiz]
469,583839,6975018,Complete: Upload and Submit a Video,"<link rel=""stylesheet"" href=""https://instructu...",True,1.0,[online_quiz]


### **Step 6. Load All Assignment Submissions from All Assignments in All Courses into List**
#### **Current Bottleneck**

In [6]:
# for each Assignment Object in the "course_assignment_objects" list, iterate over each available submission made by the user (you)

for assignment in course_assignment_objects:
    course_assignment_submissions.append(assignment.get_submission(USER_ID))

### **Step 7. Load All Submission Data into DataFrame Object**

In [7]:
submission_info = []
for submission in course_assignment_submissions:
    # only include assignments that have been completed
    try:
        if submission.attempt != None:
            submission_info.append({'assignment_id':submission.assignment_id, 'attachments':submission.attachments, 'attempt':submission.attempt, 'body':submission.body, 'due_date':submission.cached_due_date, 'grade':submission.entered_grade, 'score':submission.entered_score, 'extra_attempts':submission.extra_attempts, 'submission_id':submission.id, 'late':submission.late, 'submission_type':submission.submission_type, 'submitted_at':submission.submitted_at})
    except:
        pass
# output list of dictionaries as Pandas DataFrame
submissions = pd.DataFrame(submission_info)

# reorder columns in DataFrame
submissions = submissions[['assignment_id', 'submission_id', 'submission_type', 'body', 'attachments', 'attempt', 'extra_attempts', 'due_date', 'grade', 'score', 'late', 'submitted_at']]

# view results
submissions

,assignment_id,submission_id,submission_type,body,attachments,attempt,extra_attempts,due_date,grade,score,late,submitted_at
0,7415005,203209668,online_upload,None,[public speaking.jpg],1,None,2024-01-15T06:59:00Z,10,10.0,False,2024-01-10T18:29:04Z
1,7415009,203209924,online_upload,None,[worksheet done.pdf],1,None,2024-01-22T06:59:00Z,5,5.0,False,2024-01-22T06:04:38Z
2,7415024,203210820,online_upload,None,"[Peer Feedback Form (1).pdf, Peer Feedback For...",1,None,2024-02-05T06:59:59Z,10,10.0,False,2024-02-05T06:46:32Z
3,7415026,203210948,online_upload,None,[INTRO Feedback Form .docx.pdf],1,None,2024-01-31T06:59:00Z,10,10.0,False,2024-01-29T19:12:30Z
4,7415022,203210692,online_upload,None,[introduction speech outline.docx],1,None,2024-01-31T06:59:00Z,10,10.0,False,2024-01-31T05:53:04Z
...,...,...,...,...,...,...,...,...,...,...,...,...
243,6975017,194575904,online_quiz,"<link rel=""stylesheet"" href=""https://instructu...",[],2,None,None,3,3.0,False,2023-08-26T22:41:23Z
244,6975023,194575910,online_upload,None,[Dallin Draper.pdf],1,None,None,complete,0.0,False,2023-08-26T22:47:45Z
245,6975020,194575907,online_quiz,"<link rel=""stylesheet"" href=""https://instructu...",[],1,None,None,6,6.0,False,2023-08-26T22:59:48Z
246,6975018,194575905,online_quiz,"<link rel=""stylesheet"" href=""https://instructu...",[],1,None,None,1,1.0,False,2023-08-26T22:56:30Z


### **Output All DataFrame Objects to CSV Files**

In [8]:
courses_file = f'{USER_ID}_courses.csv'
assignments_file = f'{USER_ID}_assignments.csv'
submissions_file = f'{USER_ID}_submissions.csv'
courses.to_csv(courses_file, index=False)
course_assignments.to_csv(assignments_file, index=False)
submissions.to_csv(submissions_file, index=False)